## Determination of RMSE AND MAD
* Here, we run the DLM for each stock to determine out-of-sample forecast errors for comparison with the SGDLM.

In [27]:
import numpy as np                                          # load modules
import pandas as pd
import yfinance as yf

start_date = "2014-01-01"                                          # specifies time length
end_date = "2022-06-30"

                                                        # Entries here should be entered as desired                              
tickers = ['SBK.JO']

Close = pd.DataFrame(columns = tickers)                           # ensures that data comes in a data frame

for ticker in tickers:                                           # downloads prices of stocks from Yahoo! Finance
    Close[ticker] = yf.download(ticker, start_date, end_date, progress = False)['Close']
    
Close[2151:2152] = Close[2151:2152].multiply(np.nan) # changes entire row to nan   
print('Missing stock prices before filling gaps =', Close.isnull().sum().sum())
Close = Close.interpolate(limit_direction = 'both')                  # fills missing values using linear interpolation
print('Missing stock prices after filling gaps =', Close.isnull().sum().sum())  
print('Lenght of closing prices = ', len(Close))
print(" ")

returns = np.log(Close[tickers]).diff()      # calculates log-returns
data = returns[1:]
print("TABLE SHOWING  CLOSING PRICES")
print(Close[0:5])
print(" ")
print("TABLE SHOWING  LOG-RETURNS")
print(data[0:5])

Missing stock prices before filling gaps = 1
Missing stock prices after filling gaps = 0
Lenght of closing prices =  2162
 
TABLE SHOWING  CLOSING PRICES
             SBK.JO
Date               
2014-01-01  12942.0
2014-01-02  13031.0
2014-01-03  12853.0
2014-01-06  12630.0
2014-01-07  12886.0
 
TABLE SHOWING  LOG-RETURNS
              SBK.JO
Date                
2014-01-02  0.006853
2014-01-03 -0.013754
2014-01-06 -0.017502
2014-01-07  0.020067
2014-01-08 -0.007321


### Determination of beta

In [28]:
import math
df_range = np.linspace(0.8, 0.99, num = 20)

LLH_list = []
for beta in df_range:

   a = 0
   R =  0.0001
   c = 0.001
   r = 5

   delta = 0.999

   time_length_2 = range(782, 1288)

   LLH_comp_list = []

   for t in time_length_2:
    
      f =  a
      q = R + c

      e = data[tickers[0]][t]- f
      A = R/q
      z = (r + (e**2)/q)/(r+1)

      part_1 = np.log( math.gamma((r + 1)/2) / ( math.gamma(r/2)*(r*math.pi*q)**0.5)) 
      part_2 =  ((r+1)/2) * np.log( 1 + (e**2)/(r*q))
      LLH_comp  = part_1 - part_2
      LLH_comp_list.append(LLH_comp)

      m = a + A*e
      C = (R - A*A*q)*z
      n = r+1
      s =  c*z

      a =  m

      W =  C*(1-delta)/delta
      R = C + W

      c = s
      r = beta*n
   LLH = sum(LLH_comp_list)
   LLH_list.append(LLH)

beta = round(df_range[np.argmax(LLH_list)], 2)

print("Optimal beta = ", df_range[np.argmax(LLH_list)])    

Optimal beta =  0.95


### Determination of delta

In [29]:
import math
df_range = np.linspace(0.99, 0.999, num = 10)

LLH_list = []
for delta in df_range:

   a = 0
   R = 0.0001
   c = 0.001
   r = 5

   beta = beta
   time_length_2 = range(782, 1288)

   LLH_comp_list = []

   for t in time_length_2:
    
      f =  a
      q = R + c

      e = data[tickers[0]][t]- f
      A = R/q
      z = (r + (e**2)/q)/(r+1)

      part_1 = np.log( math.gamma((r + 1)/2) / ( math.gamma(r/2)*(r*math.pi*q)**0.5)) 
      part_2 =  ((r+1)/2) * np.log( 1 + (e**2)/(r*q))
      LLH_comp  = part_1 - part_2
      LLH_comp_list.append(LLH_comp)

      m = a + A*e
      C = (R - A*A*q)*z
      n = r+1
      s =  c*z

      a =  m

      W =  C*(1-delta)/delta
      R = C + W

      c = s
      r = beta*n

   LLH = sum(LLH_comp_list)
   LLH_list.append(LLH)

delta = round(df_range[np.argmax(LLH_list)], 3)

print("Optimal delta =", delta) 

Optimal delta = 0.999


### Determination of initial prior for the test phase 

In [30]:
beta = beta
delta = delta
time_length_2 = range(782, 1288)

a = 0
R =  0.0001
c = 0.001
r = 5

for t in time_length_2:

    f =  a
    q = R + c

    e = data[tickers[0]][t] - f
    A = R/q
    z = (r + (e**2)/q)/(r+1)

    m = a + A*e
    C = (R - A*A*q)*z
    n = r+1
    s =  c*z

    a =  m
    W = C * (1 - delta)/delta
    R = C + W
    c = s
    r = beta*n

a_0 = a
R_0 = R
c_0 = c
r_0 = r 

### Test phase

In [31]:
beta = beta
delta = delta
time_length_3 = range(1288, 2161)

a = a_0
R  = R_0
c = c_0
r = r_0

e_array = np.array([])
in_interval_values = []
for t in time_length_3:

    f =  a
    q = R + c

    e = data[tickers[0]][t] - f
    e_array = np.append(e_array, e)

    A = R/q
    z = (r + (e**2)/q)/(r+1)

    m = a + A*e
    C = (R - A*A*q)*z
    n = r+1
    s =  c*z

    a =  m
    W = C * (1 - delta)/delta
    R = C + W
    c = s
    r = beta*n

MAD = sum(np.absolute(e_array))/ len(time_length_3)
RMSE = math.sqrt( sum(e_array**2) /  len(time_length_3))
interval_coverage =  len(in_interval_values)/ len(time_length_3)*100 
print('Stock name = ',  ticker)
print("RMSE = ", RMSE)
print("MAD = ", MAD)

Stock name =  SBK.JO
RMSE =  0.023458335381630384
MAD =  0.016468692281612844
